# TP6 La factorisation LU - partie 1
## M1 informatique, Université d'Orléans 2021/2022

## 1. Retour sur l'agorithme de Canon

L'objectif de cette partie est de vous familiariser avec la représentation des matrices par blocs et également l'utilisation d'une topologie cartésienne pour lier le découpage des données à l'organisation des processus.
L'implémentation de l'algorithme de Canon pour la multiplication de matrices est disponible sur Celene. Récupérez l'archive Canon.tgz pour lire le code et l'exécuter.

Pour rappel on utilise les notations suivantes pour une matrice $A$ de taille $n\times n$ et $l<n$ avec $n$ divisible par $l$

$$A=(a_{ij})_{0<=i,j<n}=(A_{ij})_{0<=i,j<\frac{n}{l}}$$
Chaque bloc $A_{ij}$ est donc de taille $l \times l$.

On exécute ce code sur 16 processus et avec deux matrices $A$ et $B$ de taille $36 \times 36$. 

**mpirun -np 16 -oversubscribe ./main 36 0**

Pour les communications on utilise la routine *MPI_Cart_Shift* pour définir les processus avec qui des blocs de $A$ ou $B$ sont échangés.

## 2. La factorisation LU

Cette partie consiste à détailler l'algorithme de la factorisation LU. L'objectif est de comprendre la version séquentielle puis la version par blocs qui va permettre de paralléliser cette factorisation.

### 2.1 Compréhension de la version séquentielle
La factorisation LU d’une matrice carrée $A$ consiste à définir une matrice triangulaire inférieure $L$ dont la diagonale est à 1 et une matrice triangulaire supérieure $U$ telles que
$$A = LU$$

L’algorithme séquentiel classique consiste à transformer $A$ progressivement en une matrice triangulaire supérieure en utilisant le pivot de Gauss, les pivots successifs permettant de construire L.

Par exemple soit 
$A=\begin{pmatrix}
  2   &1   &3 &5\\
  4   &5   &10 &12\\
  6   &15   &30 &24\\
  4   &17   &41 &30\\
\end{pmatrix}$, on considère que $U=A$ initialement. Si on prend comme pivot $\frac{4}{2}=\frac{a_{10}}{a_{00}}$ alors 

$$\begin{pmatrix}4 &5 &10 &12\end{pmatrix} - \frac{4}{2}\begin{pmatrix}2 &1 &3 &5 \end{pmatrix}=\begin{pmatrix}0 &3 &4 &2\end{pmatrix}$$ 

On fait donc apparaître un 0 à la position (1,0) de $U$. Les matrices partiellement construites sont $L=\begin{pmatrix}
  1   &0   &0 &0\\
  2   &1   &0 &0\\
  .   &.  &. &.\\
  .   &.  &. &.\\
\end{pmatrix}$ et $U=\begin{pmatrix}
  2   &1   &3 &5\\
  0   &3  &4 &2\\
  .   &.   &. &.\\
  .   &.   &. &.\\
\end{pmatrix}$.

Pour faire apparaître un 0 à la position (2,0) et (3,0) de $U$ les pivots seront respectivement $\frac{6}{2}$ et $\frac{4}{2}$ et on aura alors

$$L=\begin{pmatrix}
  1   &0   &0 &0\\
  2   &1   &0 &0\\
  3   &.  &. &.\\
  2   &.  &. &.\\
\end{pmatrix},~~ U=\begin{pmatrix}
  2   &1   &3 &5\\
  0   &3  &4 &2\\
  0   &12   &21 &9\\
  0   &15   &55 &20\\
\end{pmatrix}$$.

$$L=\begin{pmatrix}
  1   &0   &0 &0\\
  2   &1   &0 &0\\
  3   &4  &1 &0\\
  2   &5  &3 &1\\
\end{pmatrix},~~ U=\begin{pmatrix}
  2   &1   &3 &5\\
  0   &3  &4 &2\\
  0   &0   &5 &1\\
  0   &0   &0 &7\\
\end{pmatrix}.$$


Il est à noter que pour stocker les matrices $L$ et $U$, il est possible d'utiliser qu'une seule matrice dont la partie inférieure est $L$ et la partie supérieure est $U$ avec la diagonale à 1 de $L$ non stockée.

Si on suppose que la matrice $A$ précédente de taille $4\times 4$ est répartie avec un élément par processus sachant qu'on travaille sur une topologie cartésienne de même taille que la matrice 

### 2.2 Présentation de la version par blocs

Pour la parallélisation de cette factorisation, nous allons utiliser une version par bloc de l'algorithme du pivot de gauss. Nous allons aussi faire l'hypothèse qu'il n'y aura jamais de problème de pivot non défini pour cause de division par 0.

Le principe de la factorisation LU par bloc est le suivant. Lorsque $A$ de taille $n\times n$ est découpée en blocs $A_{ij}$ de taille $l\times l$ on peut calculer la factorisation LU de $A_{00}=L_{00}U_{00}$ par l'algorithme séquentiel précédent et on peut écrire 

$$\begin{pmatrix}
  A_{00}   &A_{01} &\ldots &A_{0(l-1)}\\
  A_{10}   &A_{11} &\ldots &A_{1(l-1)}\\
  \vdots   &\vdots &\ddots &\vdots\\
  A_{(l-1)0}   &A_{(l-1)1} &\ldots &A_{(l-1)(l-1)}\\
\end{pmatrix} = \begin{pmatrix}
  L_{00}   &0 &\ldots &0\\
  L_{10}  \\
  \vdots &&I\\
  L_{(l-1)0}
\end{pmatrix} 
\times  \begin{pmatrix}
  U_{00}   &U_{01} &\ldots &U_{0(l-1)}\\
  0  &\tilde A_{11} &\ldots &\tilde A_{1(l-1)}\\
  0 &\vdots &\ddots &\vdots\\
  0 &\tilde A_{(l-1)1} &\ldots &\tilde A_{(l-1)(l-1)}\\
\end{pmatrix}$$
où $I$ désigne la matrice identité de taille $(n-l)\times (n-l)$. Les matrices $L_{00}$ et $U_{00}$ sont connues et on a
1. $L_{00}U_{0j}=A_{0j}$ pour $1\leq j <l-1$ 

Puisque $L_{00}$ est triangulaire inférieure on peut calculer $U_{0j}$ par une résolution de système.
2. $L_{i0}U_{00}=A_{i0}$ pour $1\leq i <l-1$ 

Puisque $U_{00}$ est triangulaire supérieure on peut calculer $L_{i0}$ par une résolution de système.
3. $\tilde A_{ij}+L_{i0}U_{0j}=A_{ij}$ pour $1\leq i,j < l-1$

On connaît $L_{i0}$ et $U_{0j}$ par les calculs précédents donc on peut calculer $\tilde A_{ij}$.

Q5. Quelles sont les étapes suivantes pour continuer le calcul de la factorisation LU ?
Q6. Quel est le nombre d'étapes à effectuer ?
Q7. Donnez un algorithme (en pseudo code) qui décrit les étapes du calcul.

Une première étape pour la parallélisation est de considérer que la matrice va être distribuée en respectant une topologie cartésienne. Autrement dit on va faire correspondre le nombre de blocs de la matrice avec le nombre de processus qui vont être organisés en grille 2D (comme pour l'algorithme de Canon).

On suppose qu'on dispose d'une librairie de fonctions qui permet d'effectuer des calculs matriciels dont 

- *FactLU* qui prend une matrice $A$ en paramètre et renvoie une matrice réunissant $L$ et $U$ telque que $A=L\times U$.
- *ResolutionTriangSup* qui prend en paramètre une matrice triangulaire supérieur $U$ et une matrice $A$ et qui renvoie la matrice $L$ tel que $LU=A$
- *ResolutionTriangInf* qui prend en paramètre une matrice triangulaire inférieur $L$ et une matrice $A$ et qui renvoie la matrice $U$ telque $LU=A$ 


Sur l'exemple d'une matrice $A$ de taille $36\times 36$ et une grille logique de $4\times 4$ processus 

### 2.3 Répartition de la charge
Faire l'hypothèse qu'il faut autant de processus que de blocs dans la matrice $A$ est très réductrice. Dans ce cas, l'équilibrage de charge n'est pas correcte puisque les processus vont au fur et à mesure ne plus avoir de calculs à effectuer.

Une possibilité pour lever cette hypothèse est de considérer une taille de bloc $l$ inférieur à $ \frac{n}{nbprocs}$ et de mieux répartir ces blocs sur la grille de processus. 
Si on se donne les règles suivantes 
   1. on dispose d'une grille logique de processus de taille $p \times p$ 
   2. $n$ est divisible par $p^2$
   2. la matrice $A$ de taille $n\times n$ se décompose en $b \times b$ blocs de taille $l\times l$ avec $l=\frac{n}{p^2}$ et $b=\frac{n}{l}$.
 

Pour la distribution il s'agit alors d'effectuer une distribution en round-robin comme illustrée ci-dessous.
<img src="mapping1.jpg" alt="round robin" style="width:500px;"/>

Pour notre matrice $A$ de taille $36\times 36$ il faut travailler sur une grille logique de $3 \times 3$ pour respecter les conditions de la distribution.


